# Modeling

The configured `Model` in the forge will provide predefined properties of a set of Types if may contain.

## Initialisation

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/getting-started"]))

print("The working directory is now:")
!pwd

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

# Imports

In [4]:
import json

In [5]:
from kgforge.core import Resource

In [6]:
from kgforge.specializations.resources import Dataset

## Prefixes
Prefixes are namespaces that are used to put Resource properties within a context.

In [7]:
forge.prefixes()

Used prefixes:
 prov    http://www.w3.org/ns/prov#                 
 rdf     http://www.w3.org/1999/02/22-rdf-syntax-ns#
 schema  http://schema.org/                         


## Types
The `type` property of a Resource can be associated to the available types in the Model. These types have a pre-defined set of properties.

In [8]:
forge.types()

Managed entity types:
   - Association
   - Person


## Templates
The template will provide a set of properties for the givent type that is recomended to be used when creating Resources.

### showing the properties of a type + getting the template of a Mapping for a type

In [9]:
forge.template("Person")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.
{
    type: Person
    name: hasattr
}


In [10]:
# forge.template("Person", only_required=True)

In [11]:
# forge.template("Association")

### creating (a) Resource instance(s)

#### manually (JSON)

In [12]:
forge.template("Person", output="json")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.
{
    "type": "Person",
    "name": "hasattr"
}


In [13]:
data = {
    "type": "Person",
    "name": "Jane"
}

In [14]:
resource_json = forge.from_json(data)

In [15]:
print(resource_json)

{
    type: Person
    name: Jane
}


#### programmatically (Dict)

In [16]:
template = forge.template("Person", output="dict")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.


In [17]:
template["name"] = "Jane"

In [18]:
resource_dict = forge.from_json(template)

In [19]:
print(resource_dict)

{
    type: Person
    name: Jane
}


## Validation
It is possible to verify that a Resource is compliant with the suggested type schema available in the Model.

In [20]:
jane = Resource(type="Person", name="Jane Doe")

In [21]:
john = Resource(type="Person", name="John Smith")

In [22]:
persons = [jane, john]

In [23]:
forge.validate(persons)

<count> 2
<action> _validate_one
<succeeded> True


In [24]:
jane._last_action

Action(error=None, message=None, operation='_validate_one', succeeded=True)

In [25]:
jane._validated

True

### automatic status update

In [26]:
jane.email = "jane.doe@epfl.ch"

In [27]:
jane._validated

False

### lazy actions handling

In [28]:
! cat ../../data/persons.csv

type,name,distribution
Person,Marie Curie,../../data/scientists-database/marie_curie.txt
Person,Albert Einstein,../../data/scientists-database/albert_einstein.txt


In [29]:
distribution = forge.attach("../../data/persons.csv")

In [30]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution)

In [31]:
forge.validate(jane)

<action> _validate_one
<succeeded> False
<error> ValidationError: resource has lazy actions which need to be executed before


In [32]:
# forge.validate(jane, execute_actions_before=True)

### error handling

In [33]:
mistake = Resource(type="Person")

In [34]:
resource = Resource(type="Association", agent=mistake)

In [35]:
forge.validate(resource)

<action> _validate_one
<succeeded> False
<error> ValidationError: name is missing


In [36]:
resource._last_action

Action(error='ValidationError', message='name is missing', operation='_validate_one', succeeded=False)

In [37]:
resource._validated

False